## Imports

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential

## Load and prepare the dataset



In [ ]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image

# parameters
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 1024

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load('cifar10',as_supervised = True,split = 'train')
# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)
# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load('cifar10',as_supervised = True,split = 'test')
# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)
# batch the dataset
test_dataset = test_dataset.batch(BATCH_SIZE)

## Build the Model



In [ ]:
inputs = tf.keras.layers.Input(shape = (32,32,3))
conv_1 = tf.keras.layers.Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same')(inputs)
max_pool_1 = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv_1)
conv_2 = tf.keras.layers.Conv2D(128,kernel_size = (3,3),activation = 'relu',padding = 'same')(max_pool_1)
max_pool_2 = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv_2)

bottleneck = tf.keras.layers.Conv2D(256,kernel_size =(3,3),activation = 'relu',padding = 'same')(max_pool_2)

conv_3 = tf.keras.layers.Conv2D(128,kernel_size = (3,3),activation = 'relu',padding = 'same')(bottleneck)
up_sample_1 = tf.keras.layers.UpSampling2D(size = (2,2))(conv_3)
conv_4 = tf.keras.layers.Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same')(up_sample_1)
up_sample_2 = tf.keras.layers.UpSampling2D(size = (2,2))(conv_4)

decoder_output = tf.keras.layers.Conv2D(3,kernel_size = (3,3),activation = 'sigmoid',padding = 'same')(up_sample_2)

model = tf.keras.Model(inputs = inputs,outputs = decoder_output)

model.summary()

## Configure training parameters



In [ ]:
# Please do not change the model.compile() parameters
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

## Training



In [ ]:
train_steps = len(train_dataset) // BATCH_SIZE
val_steps = len(test_dataset) // BATCH_SIZE

model.fit(train_dataset, steps_per_epoch=train_steps, validation_data=test_dataset, validation_steps=val_steps, epochs=50)

## Model evaluation



In [ ]:
result = model.evaluate(test_dataset, steps=10)

## Save the Model

In [ ]:
# Save the model you just trained
model.save("temp_model.h5")

In [ ]:
# Load the model you saved earlier
model = tf.keras.models.load_model("temp_model.h5", compile=False)

# For this assignment only. The model has to be compiled with these settings.
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

# Save the model with the compatible TF version
model.save("mymodel.h5")

In [ ]:
# You can also use this cell as a shortcut for downloading your model
from google.colab import files
files.download("mymodel.h5")